In [1]:
import geopandas as gpd
import pandas as pd

from pathlib import Path

# Standard imports
import glob
import json
import sys
import time
import uuid
import yaml

import requests
from tqdm import tqdm
import folium

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


## Set-up input parameters and directories

In [2]:
HOSPITALS_FPATH = Path("../../../data/02-raw/philippines_healthfacilities.gpkg")

PROCSSED_DIR = Path("../../../data/03-processed/")
OUTPUT_DIR = Path("../../../data/04-output/")
ISOCHRONES_DIR = PROCSSED_DIR / "isochrones"

## Utils

In [3]:
# isochrone using mapbox API
def get_mapbox_iso(df, profile, mins):
    id_processed = []
    _out = []
    _no_data = []

    with tqdm(total=df.shape[0]) as pbar:
        for uid, lat, lng in df.values:

            if not uid in id_processed:
                # mapbox URL
                iso_url = f"https://api.mapbox.com/isochrone/v1/mapbox/{profile}/{lng}%2C{lat}?contours_minutes={mins}&polygons=true&denoise=1&access_token={api_key}"
                r = requests.get(iso_url)

                if r.status_code == 200:

                    poly_gdf = gpd.read_file(json.dumps(r.json()), driver="GeoJSON")
                    poly_gdf["uid"] = uid

                    _out.append(poly_gdf)

                else:
                    print(f"{r.status_code} was returned for uid {uid}.")
                    _no_data.append(uid)

            pbar.update(1)

    # return gdf of contours
    if len(_out) != 0:
        out = pd.concat(_out)
        out.drop(
            columns=[
                "fill",
                "fillOpacity",
                "fill-opacity",
                "fillColor",
                "color",
                "opacity",
                "metric",
            ],
            inplace=True,
        )

    # return list of pois with no contour
    if len(_no_data) != 0:
        no_data = pd.concat(_no_data)
    else:
        no_data = []

    return out, no_data

## Load hospital points

In [4]:
doh_gdf = gpd.read_file(HOSPITALS_FPATH, driver="GPKG")
doh_gdf.head(2)

,id,facilityco,healthfaci,typeofheal,barangay,municipali,province,region,status,address,style,geometry
0,1.0,DOH000000000002277,Calvario Barangay Health Station,Barangay Health Station,Calvario,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.98987 6.65182)
1,2.0,DOH000000000010319,Cabunbata Barangay Health Station,Barangay Health Station,Cabunbata,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.96630 6.67152)


In [5]:
doh_gdf.shape

(23676, 12)

In [6]:
doh_gdf = doh_gdf.to_crs("epsg:4326")

In [7]:
doh_gdf["lng"] = doh_gdf["geometry"].x
doh_gdf["lat"] = doh_gdf["geometry"].y

## Filter to lacuna cities

In [8]:
# filter to target cities
target_cities = [
    "Navotas",
    "Mandaluyong",
    "Muntinlupa",
    "Dagupan City",
    "Palayan City",
    "Legazpi City",
    "Iloilo City",
    "Mandaue City",
    "Tacloban City",
    "Zamboanga City",
    "Cagayan de Oro City",
    "Davao City",
]
filtered_doh = doh_gdf[
    doh_gdf["municipali"].str.contains("|".join(target_cities), case=False)
]

In [9]:
filtered_doh["municipali"].unique()

array(['Zamboanga City', 'Cagayan De Oro City (Capital)', 'MANDAUE CITY',
       'ILOILO CITY (CAPITAL)', 'ILOILO CITY (CAPITAL)*', 'DAGUPAN CITY',
       'MANDALUYONG', 'NAVOTAS', 'MUNTINLUPA CITY', 'PALAYAN CITY',
       'DAVAO CITY', 'TACLOBAN CITY (CAPITAL)', 'LEGAZPI CITY (CAPITAL)'],
      dtype=object)

In [10]:
filtered_doh.head(2)

,id,facilityco,healthfaci,typeofheal,barangay,municipali,province,region,status,address,style,geometry,lng,lat
20,25.0,DOH000000000026388,Rio Hondo Health Center,Barangay Health Station,Rio Hondo,Zamboanga City,Zamboanga Del Sur,REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (122.08624 6.89883),122.086240,6.898832
21,26.0,DOH000000000012109,Zamboanga City Health Office (district I),Rural Health Unit,Barangay Zone III (Pob.),Zamboanga City,Zamboanga Del Sur,REGION IX (ZAMBOANGA PENINSULA),None,None,Rural Health Unit,POINT (122.07909 6.90334),122.079094,6.903339


In [11]:
filtered_doh["typeofheal"].value_counts()

Barangay Health Station                           417
Rural Health Unit                                 119
Hospital                                          110
Birthing Home/Lying-in Clinic                      68
Medical Clinic                                      5
Infirmary                                           4
Social Hygiene Clinic                               3
Drug Abuse Treatment and Rehabilitation Center      1
Name: typeofheal, dtype: int64

In [12]:
# filter to hospitals
hospitals_gdf = filtered_doh[filtered_doh["typeofheal"] == "Hospital"]

In [13]:
hospitals_gdf["municipali"].unique()

array(['Zamboanga City', 'Cagayan De Oro City (Capital)', 'MANDAUE CITY',
       'ILOILO CITY (CAPITAL)', 'DAGUPAN CITY', 'MANDALUYONG', 'NAVOTAS',
       'MUNTINLUPA CITY', 'DAVAO CITY', 'TACLOBAN CITY (CAPITAL)',
       'LEGAZPI CITY (CAPITAL)'], dtype=object)

In [14]:
hospitals_gdf.head(1)

,id,facilityco,healthfaci,typeofheal,barangay,municipali,province,region,status,address,style,geometry,lng,lat
23,28.0,DOH000000000037412,Hospital De Zamboanga,Hospital,Barangay Zone III (Pob.),Zamboanga City,Zamboanga Del Sur,REGION IX (ZAMBOANGA PENINSULA),None,None,Hospital,POINT (122.07981 6.90497),122.079813,6.904966


In [15]:
hospitals_gdf.shape

(110, 14)

In [16]:
# filter to barangay health centers
health_centers_gdf = filtered_doh[
    filtered_doh["typeofheal"] == "Barangay Health Station"
]
health_centers_gdf["municipali"].unique()

array(['Zamboanga City', 'Cagayan De Oro City (Capital)', 'MANDAUE CITY',
       'ILOILO CITY (CAPITAL)', 'ILOILO CITY (CAPITAL)*', 'DAGUPAN CITY',
       'MANDALUYONG', 'NAVOTAS', 'MUNTINLUPA CITY', 'PALAYAN CITY',
       'DAVAO CITY', 'TACLOBAN CITY (CAPITAL)', 'LEGAZPI CITY (CAPITAL)'],
      dtype=object)

In [17]:
health_centers_gdf.shape

(417, 14)

In [18]:
rhu_gdf = filtered_doh[filtered_doh["typeofheal"] == "Rural Health Unit"]
rhu_gdf.shape

(119, 14)

In [19]:
rhu_gdf

,id,facilityco,healthfaci,typeofheal,barangay,municipali,province,region,status,address,style,geometry,lng,lat
21,26.0,DOH000000000012109,Zamboanga City Health Office (district I),Rural Health Unit,Barangay Zone III (Pob.),Zamboanga City,Zamboanga Del Sur,REGION IX (ZAMBOANGA PENINSULA),None,None,Rural Health Unit,POINT (122.07909 6.90334),122.079094,6.903339
26,31.0,DOH000000000005505,Santa Catalina Main Health Center,Rural Health Unit,Santa Catalina,Zamboanga City,Zamboanga Del Sur,REGION IX (ZAMBOANGA PENINSULA),None,None,Rural Health Unit,POINT (122.08516 6.90733),122.085157,6.907329
29,34.0,DOH000000000000113,Talon-talon Main Health Center,Rural Health Unit,Talon-Talon,Zamboanga City,Zamboanga Del Sur,REGION IX (ZAMBOANGA PENINSULA),None,None,Rural Health Unit,POINT (122.11555 6.90787),122.115554,6.907869
41,48.0,DOH000000000002893,Canelar Main Health Center,Rural Health Unit,Canelar,Zamboanga City,Zamboanga Del Sur,REGION IX (ZAMBOANGA PENINSULA),None,None,Rural Health Unit,POINT (122.07065 6.91623),122.070654,6.916231
42,49.0,DOH000000000023684,Star Apple Health Center,Rural Health Unit,Baliwasan,Zamboanga City,Zamboanga Del Sur,REGION IX (ZAMBOANGA PENINSULA),None,None,Rural Health Unit,POINT (122.05729 6.91654),122.057288,6.916539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18211,19373.0,DOH000000000002034,Jacinto Rural Health Unit D,Rural Health Unit,"Jacinto, Barangay 32d",DAVAO CITY,DAVAO DEL SUR,REGION XI (DAVAO REGION),Same As 32-d Rural Health Unit,Brgy. 32-d Davao City,Rural Health Unit,POINT (125.61668 7.06894),125.616677,7.068936
18951,20125.0,DOH000000000002481,Bagiuo District Health Office,Rural Health Unit,Bagiuo District,DAVAO CITY,DAVAO DEL SUR,REGION XI (DAVAO REGION),Functional,None,Rural Health Unit,POINT (125.50960 7.12397),125.509602,7.123974
19298,20500.0,DOH000000000002383,District A (t. Claudio) Health Office,Rural Health Unit,Barangay 2-a (pob.),DAVAO CITY,DAVAO DEL SUR,REGION XI (DAVAO REGION),Functional,None,Rural Health Unit,POINT (125.60731 7.07166),125.607310,7.071657
19303,20505.0,DOH000000000006344,Davao City Health Office,Rural Health Unit,Brgy 1-a (pob),DAVAO CITY,DAVAO DEL SUR,REGION XI (DAVAO REGION),Functional,None,Rural Health Unit,POINT (125.60669 7.06388),125.606686,7.063884


## Generate Isochrones

### Get Mapbox API Key

The `secrets/mapbox.yaml` file should NOT be commited to the repo. Instead, create your own file by following this format:

`api-key: <YOUR API KEY HERE>` <br>
`mapbox-style-xyz-url: <YOUR XYZ URL HERE>`

In [20]:
SECRETS_YAML = Path("../../../secrets/mapbox.yaml")
with open(SECRETS_YAML, "r") as f:
    secrets = yaml.safe_load(f)
    api_key = secrets["api-key"]
    # MAPBOX_XYZ_URL = secrets["mapbox-style-xyz-url"]

### 5 mins. Isochrones

In [ ]:
# 5 min increment intervals
hospitals_iso_5min, no_data = get_mapbox_iso(
    hospitals_gdf[["id", "lat", "lng"]], "driving-traffic", "5"
)
hospitals_iso_10min, no_data = get_mapbox_iso(
    hospitals_gdf[["id", "lat", "lng"]], "driving-traffic", "10"
)
hospitals_iso_15min, no_data = get_mapbox_iso(
    hospitals_gdf[["id", "lat", "lng"]], "driving-traffic", "15"
)
hospitals_iso_20min, no_data = get_mapbox_iso(
    hospitals_gdf[["id", "lat", "lng"]], "driving-traffic", "20"
)
hospitals_iso_25min, no_data = get_mapbox_iso(
    hospitals_gdf[["id", "lat", "lng"]], "driving-traffic", "25"
)

In [32]:
hospitals_iso_30min, no_data = get_mapbox_iso(
    hospitals_gdf[["id", "lat", "lng"]], "driving-traffic", "30"
)

100%|██████████| 110/110 [00:24<00:00,  4.52it/s]


In [33]:
hospitals_iso_5min.to_file(
    ISOCHRONES_DIR / "iso_hospital_drivetraffic_5.geojson", driver="GeoJSON"
)
hospitals_iso_10min.to_file(
    ISOCHRONES_DIR / "iso_hospital_drivetraffic_10.geojson", driver="GeoJSON"
)
hospitals_iso_15min.to_file(
    ISOCHRONES_DIR / "iso_hospital_drivetraffic_15.geojson", driver="GeoJSON"
)
hospitals_iso_20min.to_file(
    ISOCHRONES_DIR / "iso_hospital_drivetraffic_20.geojson", driver="GeoJSON"
)
hospitals_iso_25min.to_file(
    ISOCHRONES_DIR / "iso_hospital_drivetraffic_25.geojson", driver="GeoJSON"
)
hospitals_iso_30min.to_file(
    ISOCHRONES_DIR / "iso_hospital_drivetraffic_30.geojson", driver="GeoJSON"
)

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future ver

### Get other 1min. intervals

In [21]:
# 1 min increment intervals
def get_multi_iso(iso_minutes, health_gdf=hospitals_gdf, type="hospital"):
    """minutes - max 4 contours per coordinate and 300 requests per minute"""
    facility_iso_df, no_data = get_mapbox_iso(
        health_gdf[["id", "lat", "lng"]], "driving-traffic", iso_minutes
    )
    facility_iso_df.to_file(
        ISOCHRONES_DIR / f"iso_{type}_drivetraffic_{iso_minutes}.geojson",
        driver="GeoJSON",
    )

In [23]:
# insert isochrone minutes needed
get_multi_iso("28, 29")

100%|██████████| 110/110 [00:24<00:00,  4.56it/s]
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [23]:
get_multi_iso("29, 30", health_gdf=health_centers_gdf, type="brgy_healthcenters")

  0%|          | 0/417 [00:00<?, ?it/s]

100%|██████████| 417/417 [01:20<00:00,  5.15it/s]
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [30]:
get_multi_iso("29, 30", health_gdf=rhu_gdf, type="rhu")

100%|██████████| 119/119 [00:30<00:00,  3.92it/s]
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Visualization

In [49]:
# Set map center
gdf_centroid_lat, gdf_centroid_lon = (
    hospitals_gdf.geometry.y.mean(),
    hospitals_gdf.geometry.x.mean(),
)

In [60]:
# Set basemap
map = folium.Map(
    location=[gdf_centroid_lat, gdf_centroid_lon],
    zoom_start=7,
    control_scale=True,
    tiles="cartodb positron",
)

In [55]:
test_clean = hospitals_gdf.copy()
test_clean = test_clean.drop(columns=["lat", "lng"])
test_clean = test_clean.rename(columns={"style": "type"})

In [ ]:
hospitals_iso_30min.explore(m=map, color="blue")
hospitals_iso_25min.explore(m=map, color="skyblue")
hospitals_iso_20min.explore(m=map, color="purple")
hospitals_iso_15min.explore(m=map, color="magenta")
hospitals_iso_10min.explore(m=map, color="red")
hospitals_iso_5min.explore(m=map, color="pink")

test_clean.explore(m=map, color="yellow")
folium.LayerControl(
    names=["30", "25", "20", "15", "10", "5"], position="bottomright"
).add_to(map)
map